In [22]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import cv2
from concurrent.futures import ProcessPoolExecutor
import numpy as np
from os.path import join

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning, 
    message="invalid value encountered in divide")

In [23]:
mosi_path = r'/home/drew/Desktop/Research/MMSA/datasets/CMU-MOSI/Processed/aligned_50.pkl'
with open(mosi_path, 'rb') as f:
    data = pickle.load(f)
    
train, valid, test = data.values()

In [24]:
data_root = r'/media/drew/data1/DeepLearning/MMSA/datasets/CMU-MOSI/Raw'

In [25]:
train.keys()

dict_keys(['raw_text', 'audio', 'vision', 'id', 'text', 'text_bert', 'annotations', 'classification_labels', 'regression_labels'])

In [26]:
train['id'][:5]

['03bSnISJMiM$_$11',
 '03bSnISJMiM$_$10',
 '03bSnISJMiM$_$13',
 '03bSnISJMiM$_$12',
 '03bSnISJMiM$_$1']

In [27]:
def get_data_path(data_root, id_list):
    return [join(data_root, *(id + '.mp4').split('$_$')) for id in id_list]
data_path_list = get_data_path(data_root, train['id'])
data_path_list[:5]

['/media/drew/data1/DeepLearning/MMSA/datasets/CMU-MOSI/Raw/03bSnISJMiM/11.mp4',
 '/media/drew/data1/DeepLearning/MMSA/datasets/CMU-MOSI/Raw/03bSnISJMiM/10.mp4',
 '/media/drew/data1/DeepLearning/MMSA/datasets/CMU-MOSI/Raw/03bSnISJMiM/13.mp4',
 '/media/drew/data1/DeepLearning/MMSA/datasets/CMU-MOSI/Raw/03bSnISJMiM/12.mp4',
 '/media/drew/data1/DeepLearning/MMSA/datasets/CMU-MOSI/Raw/03bSnISJMiM/1.mp4']

In [29]:
# 定义一个函数来处理单个视频文件
def process_video(file_path):
    # 初始化一个列表来保存RGB帧数据
    rgb_frames = []
    
    # 使用OpenCV打开视频文件
    cap = cv2.VideoCapture(file_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # 如果没有帧了，跳出循环
        
        # 转换颜色空间从BGR到RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frames.append(frame_rgb)
    
    cap.release()
    return np.array(rgb_frames)

# 对数据路径列表使用并行处理
def parallel_process_videos(data_path_list):
    with ProcessPoolExecutor() as executor:
        # 使用map函数并行处理所有视频文件
        results = executor.map(process_video, data_path_list)
    
    # 将结果整合并返回
    return list(results)

video_frame = parallel_process_videos(data_path_list[:5])

In [33]:
video_frame[3].shape

(81, 360, 640, 3)